# Chapter 3: Techniques for Mitigating Hallucinations in Large Language Models (LLMs)

This chapter covers several techniques to reduce hallucinations in Large Language Models (LLMs). By implementing these methods, we can help models generate responses that are more accurate and grounded in factual information.

---

## Learning Objectives

By the end of this chapter, you will be able to:
- Apply prompt engineering techniques to guide LLMs toward more accurate responses.
- Implement retrieval-based grounding to provide factual context to LLM outputs.
- Understand the benefits of fine-tuning models with domain-specific data to reduce hallucinations.

---

## 1. Prompt Engineering

**Prompt engineering** involves crafting input prompts in a way that guides the model to produce more accurate responses. By making prompts clear and context-rich, we can reduce the likelihood of hallucination.

### Example: Crafting Effective Prompts

Let’s try an example of how slight changes in prompt wording can impact the model’s response.

---

### Code Implementation: Prompt Engineering

In this example, we’ll compare two different prompts and observe how GPT-4 and LLaMA respond to each.


In [ ]:
import openai
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
# Set up OpenAI API for GPT-4
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your OpenAI API key

def gpt4_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']


In [ ]:
# Set up LLaMA model
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b")
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b")

def llama_response(prompt):
    inputs = llama_tokenizer(prompt, return_tensors="pt")
    outputs = llama_model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
    return llama_tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
# Define prompts
basic_prompt = "Describe Mars."
detailed_prompt = "Describe the planet Mars, including its atmosphere, surface features, and any recent exploration missions."

In [ ]:
# Test with GPT-4o
print("GPT-4o Response to Basic Prompt:")
print(gpt4_response(basic_prompt))
print("\nGPT-4 Response to Detailed Prompt:")
print(gpt4_response(detailed_prompt))


In [ ]:
# Test with LLaMA
print("\nLLaMA Response to Basic Prompt:")
print(llama_response(basic_prompt))
print("\nLLaMA Response to Detailed Prompt:")
print(llama_response(detailed_prompt))

---

## Observations

After running the code above, compare the responses to each prompt:

1. **Detail and Specificity**: Does the detailed prompt provide a more accurate and context-rich response?
2. **Reduction in Hallucination**: Observe if the detailed prompt helps to reduce any fabricated information about Mars.
3. **Differences Between Models**: Note if GPT-4 or LLaMA handles the prompt variations differently.

---

## 2. Retrieval-Based Grounding

**Retrieval-based grounding** is a technique where we provide the model with specific, factual information before generating a response. This approach is useful for reducing hallucination by grounding the model’s response in external data.

### Example: Implementing Retrieval-Based Grounding

In this example, we’ll simulate retrieval-based grounding by using a basic knowledge base and retrieving relevant information to provide context.

---

### Code Implementation: Retrieval-Based Grounding


In [ ]:
# Simulated knowledge base with factual information
knowledge_base = {
    "Mars": "Mars is the fourth planet from the Sun. It has a thin atmosphere composed mostly of carbon dioxide.",
    "solar system": "The Solar System consists of the Sun and the celestial bodies that orbit it, including planets, moons, and asteroids.",
    "NASA": "NASA is the United States' space agency, responsible for the nation’s civilian space program and for aeronautics and aerospace research."
}



In [ ]:
def grounded_response(query):
    # Search for relevant information in the knowledge base
    for key, info in knowledge_base.items():
        if key.lower() in query.lower():
            return f"{info}\n\nAdditional response: The model can add context based on this factual grounding."
    return "No relevant information found in the knowledge base."



In [ ]:
# Example queries
query_1 = "Tell me about Mars."
query_2 = "What is NASA's role in space exploration?"

# Generate grounded responses
print("Grounded Response for Query 1:")
print(grounded_response(query_1))
print("\nGrounded Response for Query 2:")
print(grounded_response(query_2))


---

## Exercise: Building a Basic Knowledge Base

### Instructions

1. **Expand the Knowledge Base**: Add more entries to `knowledge_base` with factual information about topics like "Jupiter," "Milky Way," or "Apollo missions."
2. **Test with Additional Queries**: Run the `grounded_response` function with new queries to see how well it retrieves relevant information and grounds the model’s responses.

---

## 3. Fine-Tuning the Model

Fine-tuning an LLM with domain-specific data is another powerful technique to reduce hallucinations. When a model is fine-tuned on targeted, high-quality data, it becomes more specialised in that domain and is less likely to produce incorrect information.

### Example: Explanation of Fine-Tuning Process

To fine-tune a model, we need a dataset that’s representative of the target domain. We then train the model further on this data with lower learning rates and fewer epochs than initial training. This helps it adapt to new information without overfitting.

> **Note**: For this notebook, we won’t perform fine-tuning directly due to resource constraints, but you can refer to the code template below as a guide.

---

### Code Template for Fine-Tuning with Transformers


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import torch



In [ ]:
# Load your dataset
# Assuming `dataset.csv` contains columns 'input_text' and 'target_text' for fine-tuning
data = pd.read_csv("path/to/your/dataset.csv")


In [ ]:
# Prepare dataset
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b")
inputs = tokenizer(list(data['input_text']), return_tensors="pt", padding=True, truncation=True)
outputs = tokenizer(list(data['target_text']), return_tensors="pt", padding=True, truncation=True)
dataset = torch.utils.data.TensorDataset(inputs["input_ids"], outputs["input_ids"])


In [ ]:
# Fine-tuning setup
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b")
training_args = TrainingArguments(output_dir="./results", num_train_epochs=3, per_device_train_batch_size=2)



In [ ]:
# Trainer
trainer = Trainer(model=model, args=training_args, train_dataset=dataset)
trainer.train()

# Save fine-tuned model
model.save_pretrained("./fine_tuned_model")

---

## Summary

In this chapter, we covered several techniques to mitigate hallucinations in LLMs:

1. **Prompt Engineering**: Crafting prompts carefully to guide model responses.
2. **Retrieval-Based Grounding**: Providing factual context to help ground the model’s output.
3. **Fine-Tuning**: Training the model further on specific data to reduce hallucinations in that domain.

Using these techniques can significantly improve the accuracy and reliability of LLM responses, making them more suitable for applications where factual correctness is critical.
